<a href="https://colab.research.google.com/github/pi-zz-a/Text-Generation/blob/script-buddy/friends_script_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 34.1MB/s 
     |████████████████████████████████| 1.1MB 38.0MB/s 
     |████████████████████████████████| 890kB 45.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bd97ece860aacc5b944eb7f1f56f6957f1b3acf6cf3d2e35d28d2d09a7a92a7c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import torch
import math
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import os
import random
from language_modelling import ScriptData

In [0]:
torch.cuda.empty_cache()

In [0]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
output_dir='/content/drive/My Drive/models'

In [0]:
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

In [8]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [0]:
torch.cuda.empty_cache()

In [46]:
model = model.to(device)
ppls = []
for f in os.listdir('test'):
  if 'gpt2' not in f and 'check' not in f:
    pp = 0
    sum_loss=0
    script_count=0
    print(f)
    dataset = ScriptData(tokenizer= tokenizer, file_path=os.path.join("test",f))
    script_loader = DataLoader(dataset,batch_size=1,shuffle=True)
    for idx, script in enumerate(script_loader):
        outputs = model(script.to(device), labels=script.to(device))
        loss, logits = outputs[:2]
        sum_loss = sum_loss + loss.detach().data
        script_count = script_count + 1
    pp = math.pow(2,sum_loss/script_count)
    ppls.append(pp)
    print(f,pp)

84.txt
84.txt 5.220406635158218
85.txt
85.txt 5.096366200403334
83.txt
83.txt 4.374482880332669
